In [48]:
from glob import glob
from pyproj import CRS
import xarray as xr
from grids import georef_netcdf_rioxarray
from tqdm import tqdm
from pathlib import Path

def decode_satellite_view_angle(view_angle_data_array: xr.DataArray) -> xr.DataArray:
    # # Encoded with factor 2
    # satellite_view_angle = satellite_view_angle / 2
    # # We consider that if there are no data we are outside the swath and if we are outside the swath it means that VZA are generally high and cannot be as low as 1

    view_angle = view_angle_data_array
    if (view_angle == 0).sum() > 0 and (view_angle == 1).sum() == 0:
        view_angle = view_angle.where(view_angle > 0, 255)
    view_angle_data_array[:] = view_angle /2
    return view_angle_data_array

platform= "JPSS2"
sza_files = sorted(glob(f'/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/{platform}/produit_synopsis/*/*/*SatelliteZenithAngle.tif'))
fsc_files = sorted(glob(f'/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/{platform}/no_forest_red_band_screen/*/*/*no_forest_red_band_screen.tif'))
for sza_file in tqdm(sza_files):

    fsc_file = [f for f in fsc_files if Path(sza_file).name[:15] in f]
    if len(fsc_file)==1:
        fsc_file = fsc_file[0]
    else:
        continue
    fsc = xr.open_dataarray(fsc_file)
    sza = xr.open_dataset(sza_file).data_vars['band_data']
    sza = decode_satellite_view_angle(sza)
    sza = sza.where(fsc!=230, 255).astype('u1')
    sza.rio.to_raster(sza_file.replace('SatelliteZenithAngle', 'SatelliteZenithAngleMod').replace(f'{platform}/produit_synopsis', f'{platform}/no_forest_red_band_screen'))

100%|██████████| 240/240 [01:00<00:00,  3.97it/s]


In [21]:
xr.open_dataset('/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/SNPP/produit_synopsis/2023/10/20231001_123430_npp_lzw_SatelliteZenithAngle.tif').data_vars['band_data']

<xarray.DataArray 'band_data' (band: 1, y: 3112, x: 4447)> Size: 111MB
[13839064 values with dtype=float64]
Coordinates:
  * band     (band) int64 8B 1
  * x        (x) float64 36kB -5.002 -4.998 -4.995 -4.992 ... 9.995 9.998 10.0
  * y        (y) float64 25kB 51.49 51.49 51.49 51.48 ... 41.01 41.01 41.0 41.0
Attributes:
    AREA_OR_POINT:  Area
    grid_mapping:   spatial_ref

In [2]:
from glob import glob
sza_files = sorted(glob('/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/SNPP/produit_synopsis/*/*/*SatelliteZenithAngle.tif'))
fsc_files = sorted(glob('/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/SNPP/no_forest_red_band_screen/*/*/*no_forest_red_band_screen.tif'))

In [16]:
import xarray as xr
(xr.open_dataarray(fsc_files[3]) == 230).sum()

<xarray.DataArray 'band_data' ()> Size: 8B
array(0)
Coordinates:
    spatial_ref  int64 8B ...

In [6]:
ls /home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/SNPP/produit_synopsis/*/*

/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_rejeu/SNPP/produit_synopsis/2023/10:
20231001_123430_npp_lzw_cc_dist_mask.tif
20231001_123430_npp_lzw_CCNPPJSNOW_mask.tif
20231001_123430_npp_lzw_CCNPPJSNOW_reproj.tif
20231001_123430_npp_lzw_cloud.tif
20231001_123430_npp_lzw_cloud.tif.aux.xml
20231001_123430_npp_lzw_fsc.tif
20231001_123430_npp_lzw_ndsi.tif
20231001_123430_npp_lzw_SatelliteZenithAngleMod.tif
20231001_123430_npp_lzw_SatelliteZenithAngle.tif
20231001_123430_npp_produit_synopsis.tif
20231002_121558_npp_lzw_cc_dist_mask.tif
20231002_121558_npp_lzw_CCNPPJSNOW_mask.tif
20231002_121558_npp_lzw_CCNPPJSNOW_reproj.tif
20231002_121558_npp_lzw_cloud.tif
20231002_121558_npp_lzw_fsc.tif
20231002_121558_npp_lzw_ndsi.tif
20231002_121558_npp_lzw_SatelliteZenithAngleMod.tif
20231002_121558_npp_lzw_SatelliteZenithAngle.tif
20231002_121558_npp_produit_synopsis.tif
20231002_135640_npp_lzw_cc_dist_mask.tif
20231002_135640_npp_lzw_CCNPPJSNOW_mask.tif
20231002_135640_npp_lzw_CCNPPJSNOW_repro

In [15]:
sza.to_netcdf(f.replace('SatelliteZenithAngle', 'SatelliteZenithAngleMod').replace('JPSS1', 'JPSS1/no_forest_red_band_screen'), encoding={'band_data': {"zlib": True}})

In [ ]:
import xarray as xr
from grids import georef_netcdf
import pyproj

def georef_netcdf(data_array: xr.DataArray | xr.Dataset, crs: pyproj.CRS) -> xr.Dataset | xr.Dataset:
    """
    The strict minimum to georeference in netCDF convention

    Enforce georeferencing for GDAL/QGIS
    https://github.com/pydata/xarray/issues/2288
    https://gis.stackexchange.com/questions/230093/set-projection-for-netcdf4-in-python
    """

    # dims = dim_name(crs=crs)
    data_array.coords["y"].attrs["axis"] = "Y"
    data_array.coords["x"].attrs["axis"] = "X"
    data_array.attrs["grid_mapping"] = "spatial_ref"

    georeferenced = data_array.assign_coords(coords={"spatial_ref": 0})
    georeferenced.coords["spatial_ref"].attrs["spatial_ref"] = crs.to_wkt()

    return georeferenced

fn = '/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/snowCoverComposite_noaa20_20250520_1200_00000_testlogr8220x.nc'
ds = xr.open_dataset(fn)
georef_netcdf(data_array=ds, crs=pyproj.CRS.from_epsg(4326)).to_netcdf(fn.replace('.nc', '_georef.nc'))

In [7]:
from view_angle_daily_composite import create_temporal_composite_meteofrance

compo = create_temporal_composite_meteofrance(daily_snow_cover_files=
                                      ['/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/frSnowCoverTif_noaa20_20250520_1137_38876_testlogr8220x.tif',
                                       '/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/frSnowCoverTif_noaa20_20250520_1305_38877_testlogr8220x.tif'
                                       ],
                                       daily_geometry_files=
                                       ['/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/szaSnowCoverTif_noaa20_20250520_1124_38876_testlogr8220x.tif',
                                        '/home/imperatoren/work/VIIRS_S2_comparison/data/CMS_samples/sample_composite_journalier/20231202_131154_npp_lzw_SatelliteZenithAngle_FAUX.tif'])

In [8]:
compo.to_netcdf('test_compo_20250520.nc')